In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, Input

train_path = "/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/train.csv"
test_path = "/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/test.csv"
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print(f"Train Shape: {train_df.shape}, Test Shape: {test_df.shape}")
print(train_df.describe().T)

# ========== Step 2: Preprocessing ==========
X = train_df.iloc[:, :55].values  # 5 composition + 50 component properties
Y = train_df.iloc[:, 55:].values  # 10 blend properties
X_test = test_df.iloc[:, 1:].values  # skip ID

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
X_test_scaled = scaler_X.transform(X_test)

scaler_Y = StandardScaler()
Y_scaled = scaler_Y.fit_transform(Y)

# ========== Step 3: Residual ANN Model ==========
def mape_loss(y_true, y_pred):
    diff = tf.abs((y_true - y_pred) / tf.clip_by_value(tf.abs(y_true), 1e-8, tf.float32.max))
    return tf.reduce_mean(diff)

def residual_block(x, units):
    shortcut = x
    x = layers.Dense(units, activation='relu')(x)
    x = layers.LayerNormalization()(x)
    x = layers.Dense(units)(x)
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

def build_model(input_dim):
    inputs = Input(shape=(input_dim,))
    x = layers.Dense(256, activation='relu')(inputs)
    x = residual_block(x, 256)
    x = layers.Dropout(0.3)(x)
    x = residual_block(x, 256)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(10)(x)
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=mape_loss, metrics=[mape_loss])
    return model

# ========== Step 4: Train/Validation Split ==========
X_train, X_val, Y_train, Y_val = train_test_split(X_scaled, Y_scaled, test_size=0.1, random_state=42)

model = build_model(X_train.shape[1])
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=300,
    batch_size=64,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

# ========== Step 5: Validation MAPE ==========
val_pred = model.predict(X_val)
val_true = scaler_Y.inverse_transform(Y_val)
val_pred_inv = scaler_Y.inverse_transform(val_pred)
val_mape = mean_absolute_percentage_error(val_true, val_pred_inv)
print(f"📉 Validation MAPE: {val_mape:.5f}")

# ========== Step 6: Retrain on Full Dataset (team_size = 0) ==========
model_full = build_model(X_scaled.shape[1])
model_full.fit(
    X_scaled, Y_scaled,
    epochs=int(len(history.history['loss'])),  # same as before
    batch_size=64,
    verbose=1
)

# ========== Step 7: Test Predictions ==========
Y_test_scaled = model_full.predict(X_test_scaled)
Y_test_final = scaler_Y.inverse_transform(Y_test_scaled)

# ========== Step 8: Submission ==========
submission = pd.DataFrame(Y_test_final, columns=[f'BlendProperty{i+1}' for i in range(10)])
submission.insert(0, 'ID', test_df['ID'])
submission.to_csv("submission_fuelblendnet.csv", index=False)
print("✅ Submission saved as `submission_fuelblendnet.csv`")


Train Shape: (2000, 65), Test Shape: (500, 56)
                      count      mean       std       min       25%       50%  \
Component1_fraction  2000.0  0.180690  0.163200  0.000000  0.030000  0.140000   
Component2_fraction  2000.0  0.182910  0.163704  0.000000  0.040000  0.150000   
Component3_fraction  2000.0  0.179820  0.166283  0.000000  0.020000  0.140000   
Component4_fraction  2000.0  0.342090  0.141119  0.010000  0.220000  0.350000   
Component5_fraction  2000.0  0.114490  0.080219  0.000000  0.050000  0.120000   
...                     ...       ...       ...       ...       ...       ...   
BlendProperty6       2000.0 -0.003497  1.009126 -2.808210 -0.697379 -0.011649   
BlendProperty7       2000.0 -0.013568  1.000613 -2.994571 -0.622453  0.133470   
BlendProperty8       2000.0 -0.017236  0.998759 -3.621080 -0.725564 -0.001548   
BlendProperty9       2000.0 -0.001507  1.001096 -3.292727 -0.702384 -0.002604   
BlendProperty10      2000.0 -0.001795  0.990433 -2.476429 -0.7